## Goal
We want to explore how better is the performance of the system of generating a draft with a multiple step process rather than a single step process

### Initial Variables

In [1]:
import sys 
import os
import json_tricks as json
import time
sys.path.append("../../")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Generater import Generater
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import Email


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
generator = Generater(ollama_client, anyllm_client)
path_to_enquiry = "../../dataset/fake_email_1.json"
with open(path_to_enquiry, "r") as f:
    enquiry = json.load(f)

enquiry_text = enquiry["email"]
subject = enquiry_text.split("\n\n")[0]
body = '\n'.join(enquiry_text.split("\n\n")[1:])

Performing health check on http://localhost:11434
Ollama is running
Health check passed


### Single Step Process

In [2]:

def single_shot_prompt(email: Email):
    prompt = f""" You are David Frischer, the program administrator for the UCL Software Engineering program.
            You have received the following enquiry from a student:
            Subject: {email.subject}
            Body: {email.body}
            
           
            Draft a response to the student's email.
            """
    workspace_slug =  anyllm_client.get_workspace_slug("General")
    thread = anyllm_client.new_thread(workspace_slug, "tmp-thread")
    thread_slug = thread["slug"]
    start_time = time.time()
    response = anyllm_client.chat_with_thread(workspace_slug, thread_slug, prompt)
    total_time = time.time() - start_time
    anyllm_client.delete_thread(workspace_slug, thread_slug)
    return response, total_time

### Response format

## Mutiple Step Process

In [4]:
def multi_step_process(email:Email):
    start_time = time.time()
    response = generator.single_run_reply_to_email(email)
    response_time = time.time() - start_time
    
    return response, response_time
    

## Generate for multiple drafts

In [7]:

path_to_enquiries = "../../dataset"

counter = 0
results_folder = "./results"
os.makedirs(results_folder, exist_ok=True)
        
for enquiry_path in os.listdir(path_to_enquiries):
    filepath = os.path.join(results_folder, f"results_{enquiry_path}")
    results = {}
    try:
        if not enquiry_path.endswith(".json") or not "fake" in enquiry_path:
            continue
        print(f"Processing {enquiry_path}")
        with open(os.path.join(path_to_enquiries, enquiry_path), "r") as f:
            enquiry = json.load(f)
            
        email = enquiry["email"]
        if "subject:" in email.lower():
            subject = email.split("\n")[0].replace("Subject:", "").strip()
            body = "\n".join(email.split("\n")[1:])
        else:
            subject = "Subject not found"
            body = email
        
        email = Email(subject=subject, body=body, id=counter)
        counter += 1
        
        response_single, response_time_single = single_shot_prompt(email)
        response_multi, response_time_multi = multi_step_process(email)
        
        results[str(email.id)] = {"single_shot": {"response": response_single, "response_time": response_time_single},
                                  "multi_step": {"response": response_multi, "response_time": response_time_multi}}

        with open(filepath, "w") as f:
            json.dump(results, f)
    except Exception as e:
        print(f"Error processing {enquiry_path}")
        print(e)
        continue    

    
print(results)


Processing fake_email_10.json
Creating new thread with name: tmp-thread in workspace: general


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: test123 in workspace: general
Total Sim Distance: 0.369832381606102
Total Sim Distance: 0.4092678874731064


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Processing fake_email_3.json
Creating new thread with name: tmp-thread in workspace: general


...

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: test123 in workspace: general
Total Sim Distance: 0.5052822977304459
Total Sim Distance: 0.3734423965215683


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Processing example.json
Error processing example.json
Expecting value: line 4 column 12 (char 87)
Processing fake_email_17.json
Creating new thread with name: tmp-thread in workspace: general


KeyboardInterrupt: 